# Supervised Finetuning Dataset

语言模型的输入是 token 序列, 对于有监督学习(Supervised FineTuning,SFT)任务可以将语言任务数据转化为通用的问答（QA）模式，SFT学习时仅拟合回答(A)

所以有监督任务数据面临（1）将有监督任务数据转化为通用的 QA (2) 构造有监督学习的 label

1. message format
2. dataset
3. data collactor function

处理流程

1. SFT 数据为 QA 对, 考虑多轮对话情形, 使用 chat message 组织对话，使用自定义对话模版
2. 将单条数据 tokenizer 化
3. 编写 collate function

## 数据 & Tokenizer

In [1]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('Qwen/Qwen3-0.6B', 
                                          #local_dir='~/.cache/huggingface/', # 如果可以直连 huggingface, 去除此行.
                                         )

In [2]:
DEFINIED_SYSTEM_PROMPT='你是小冬瓜智能体,请安全详细回答用户 USER 的问题'
messages_1=[
    {'role':'SYSTEM', 'content':DEFINIED_SYSTEM_PROMPT},
    {'role':'USER', 'content':'$sin^2x+cos^2x=?'},
    {'role':'ASSISTANT', 'content':'结果为 $\\boxed{1}$'},
    {'role':'USER', 'content':'为什么?'},
    {'role':'ASSISTANT', 'content':'在单位圆上的任意一点与原点的连线,该线在 xy 轴上的投影分别为$sinx,cosx$,根据勾股定理可证$sin^2x+cos^2x=1$'},
]

messages_2=[    
    {'role':'SYSTEM', 'content':DEFINIED_SYSTEM_PROMPT},
    {'role':'USER', 'content':'什么是人工智能?'},
    {'role':'ASSISTANT', 'content':'人工智能是让机器模拟人类思维的技术。'},
]
messages_3=[    
    {'role':'SYSTEM', 'content':DEFINIED_SYSTEM_PROMPT},
    {'role':'USER', 'content':'如何计算复利?'},
    {'role':'ASSISTANT', 'content':'复利计算公式：本息和 = 本金 × (1 + 利率)^期数。'},
]
messages_4=[    
    {'role':'SYSTEM', 'content':DEFINIED_SYSTEM_PROMPT},
    {'role':'USER', 'content':'“哈基米”翻译成英文'},
    {'role':'ASSISTANT', 'content':'“哈基米”翻译成英文通常是 "Hakimi"（人名音译）。'},
]
messages_list = [messages_1, messages_2, messages_3, messages_4]

官方 tokenizer 会自带 chat template 模版, 以下代码仅作示例, 我们将自定义实现一个模版函数

In [18]:
# print(tokenizer.chat_template)  # 可查看模版描述

messages_tmp=[    
    {'role':'system', 'content':DEFINIED_SYSTEM_PROMPT},
    {'role':'user', 'content':'“哈基米”翻译成英文'},
    {'role':'assistant', 'content':'“哈基米”翻译成英文通常是 "Hakimi"（人名音译）。'},
]

prompt = tokenizer.apply_chat_template(messages_tmp, 
                              tokenize=False)
print(prompt)

<|im_start|>system
你是小冬瓜智能体,请安全详细回答用户 USER 的问题<|im_end|>
<|im_start|>user
“哈基米”翻译成英文<|im_end|>
<|im_start|>assistant
<think>

</think>

“哈基米”翻译成英文通常是 "Hakimi"（人名音译）。<|im_end|>



## Chat Template

对话模版有两种组织方式：

1. 对文本进行格式化处理
2. 对 token id 进行处理

### 方式1

In [4]:
def ChatTemplate(example):
    prompt = '<SOS>'
    for i, item in enumerate(example):
        prompt += '#' + item['role'] + ':' + item['content'] 
        if i % 2 == 0 and i != 0:
            prompt += '<EOS>' # 补全回答都要 EOS
    return prompt
    
prompt = ChatTemplate(messages_1)
print(prompt)

<SOS>#SYSTEM:你是小冬瓜智能体,请安全详细回答用户 USER 的问题#USER:$sin^2x+cos^2x=?#ASSISTANT:结果为 $\boxed{1}$<EOS>#USER:为什么?#ASSISTANT:在单位圆上的任意一点与原点的连线,该线在 xy 轴上的投影分别为$sinx,cosx$,根据勾股定理可证$sin^2x+cos^2x=1$<EOS>


In [5]:
import torch

input_id = tokenizer( [prompt], add_special_tokens=True ) # tokenizer 默认输入列表
print(input_id['input_ids'])

decode_prompt = tokenizer.decode(input_id['input_ids'][0], skip_special_tokens=False)
print(decode_prompt)

[[18858, 3126, 61125, 46487, 25, 105043, 30709, 99949, 100857, 100168, 31914, 11, 14880, 99464, 100700, 102104, 20002, 13872, 43589, 86119, 2, 6448, 21701, 15940, 61, 17, 87, 10, 9407, 61, 17, 87, 19884, 2, 4939, 3846, 2821, 25, 59151, 17714, 57960, 79075, 90, 16, 31716, 27, 55940, 61125, 6448, 25, 100678, 30, 2, 4939, 3846, 2821, 25, 18493, 75317, 100213, 101913, 108112, 100380, 57218, 52129, 27442, 9370, 116539, 11, 75882, 43268, 18493, 30784, 8908, 121, 112, 101913, 111367, 105706, 3, 15940, 87, 11, 9407, 87, 54876, 100345, 105170, 99223, 22382, 21887, 30440, 33477, 3, 15940, 61, 17, 87, 10, 9407, 61, 17, 87, 28, 16, 3, 27, 55940, 29]]
<SOS>#SYSTEM:你是小冬瓜智能体,请安全详细回答用户 USER 的问题#USER:$sin^2x+cos^2x=?#ASSISTANT:结果为 $\boxed{1}$<EOS>#USER:为什么?#ASSISTANT:在单位圆上的任意一点与原点的连线,该线在 xy 轴上的投影分别为$sinx,cosx$,根据勾股定理可证$sin^2x+cos^2x=1$<EOS>


1. 有监督学习任务，目标是拟合 Assistant 内容, 其损失函数与 pre-trained 时使用的 Cross-Entropy Loss 一致
2. 分析以上处理方法，tokenize 字符串时, 就需要在 token id 序列中找到 ASSISTANT 的序列内容

截取回答内容需要定位到:

1. 头: #ASSISTANT 最后一个 token
2. 尾: `<EOS>`

代码省略在列表查找子列表位置问题

In [6]:
ids = tokenizer('#ASSISTANT')['input_ids']
print(ids)
for i in ids:
    print(tokenizer.decode(i))

ids = tokenizer('<EOS>')['input_ids'] #并非按照我们期望encode成一个 token id
print(ids)
for i in ids:
    print(tokenizer.decode(i)) 

[2, 4939, 3846, 2821]
#
ASS
IST
ANT
[23835, 3126, 29]
<E
OS
>


### Tokenizer 分析

Tokenizer 会预设专用的 token，以 Qwen3 举例, 有 eos_token `<|im_end|>`, 但是句子开头在`'additional_special_tokens':` 上的 `<|im_start|>`

较为特殊的是有 `'pad_token': '<|endoftext|>',`, 对应的词元写法应当为'<|pad|>'更加合适, 本 lc 不进行过度修改

In [7]:
tokenizer.special_tokens_map

{'eos_token': '<|im_end|>',
 'pad_token': '<|endoftext|>',
 'additional_special_tokens': ['<|im_start|>',
  '<|im_end|>',
  '<|object_ref_start|>',
  '<|object_ref_end|>',
  '<|box_start|>',
  '<|box_end|>',
  '<|quad_start|>',
  '<|quad_end|>',
  '<|vision_start|>',
  '<|vision_end|>',
  '<|vision_pad|>',
  '<|image_pad|>',
  '<|video_pad|>']}

In [8]:
DEFINED_EOS_TOKEN = '<|im_end|>'
DEFINED_SOS_TOKEN = '<|im_start|>'
DEFINED_PAD_TOKEN = '<|endoftext|>'

def ChatTemplateDefinedToken(example):
    prompt = DEFINED_SOS_TOKEN
    for i, item in enumerate(example):
        prompt += '\n#' + item['role'] + ':' + item['content'] 
        if i % 2 == 0 and i != 0:
            prompt += DEFINED_EOS_TOKEN # 补全回答都要 EOS
    return prompt

In [9]:
prompt = ChatTemplateDefinedToken(messages_1)
print(prompt, '\n\n')

input_id = tokenizer( [prompt], add_special_tokens=True ) # tokenizer 默认输入列表
print(input_id['input_ids'], '\n\n')

decode_prompt = tokenizer.decode(input_id['input_ids'][0], skip_special_tokens=False)
print(decode_prompt, '\n\n')

<|im_start|>
#SYSTEM:你是小冬瓜智能体,请安全详细回答用户 USER 的问题
#USER:$sin^2x+cos^2x=?
#ASSISTANT:结果为 $\boxed{1}$<|im_end|>
#USER:为什么?
#ASSISTANT:在单位圆上的任意一点与原点的连线,该线在 xy 轴上的投影分别为$sinx,cosx$,根据勾股定理可证$sin^2x+cos^2x=1$<|im_end|> 


[[151644, 198, 2, 46487, 25, 105043, 30709, 99949, 100857, 100168, 31914, 11, 14880, 99464, 100700, 102104, 20002, 13872, 43589, 86119, 198, 2, 6448, 21701, 15940, 61, 17, 87, 10, 9407, 61, 17, 87, 28, 5267, 2, 4939, 3846, 2821, 25, 59151, 17714, 57960, 79075, 90, 16, 31716, 151645, 198, 2, 6448, 25, 100678, 5267, 2, 4939, 3846, 2821, 25, 18493, 75317, 100213, 101913, 108112, 100380, 57218, 52129, 27442, 9370, 116539, 11, 75882, 43268, 18493, 30784, 8908, 121, 112, 101913, 111367, 105706, 3, 15940, 87, 11, 9407, 87, 54876, 100345, 105170, 99223, 22382, 21887, 30440, 33477, 3, 15940, 61, 17, 87, 10, 9407, 61, 17, 87, 28, 16, 3, 151645]] 


<|im_start|>
#SYSTEM:你是小冬瓜智能体,请安全详细回答用户 USER 的问题
#USER:$sin^2x+cos^2x=?
#ASSISTANT:结果为 $\boxed{1}$<|im_end|>
#USER:为什么?
#ASSISTANT:在单位圆上的任意

可以想办法在 label 前增加一个 special token, 用于定位 label 的开始位置。

In [10]:
DEFINED_LABEL_START_TOKEN = '<|box_start|>'


def ChatTemplateForLabel(example):
    prompt = DEFINED_SOS_TOKEN
    for i, item in enumerate(example):
        prompt += '\n#' + item['role'] + ':' 
        if item['role'] == 'ASSISTANT':
            prompt += DEFINED_LABEL_START_TOKEN
        prompt += item['content'] 
        if i % 2 == 0 and i != 0:
            prompt += DEFINED_EOS_TOKEN # 补全回答都要 EOS
    return prompt

print(tokenizer(DEFINED_LABEL_START_TOKEN))

prompt = ChatTemplateForLabel(messages_1)
print('\n\n[format prompt]:\n', prompt, )

input_id = tokenizer( [prompt], add_special_tokens=True ) # tokenizer 默认输入列表
print('\n\n[format prompt input_id]:\n',input_id['input_ids'])

decode_prompt = tokenizer.decode(input_id['input_ids'][0], skip_special_tokens=False)
print('\n\n[format prompt input_id decode]:\n',decode_prompt)

{'input_ids': [151648], 'attention_mask': [1]}


[format prompt]:
 <|im_start|>
#SYSTEM:你是小冬瓜智能体,请安全详细回答用户 USER 的问题
#USER:$sin^2x+cos^2x=?
#ASSISTANT:<|box_start|>结果为 $\boxed{1}$<|im_end|>
#USER:为什么?
#ASSISTANT:<|box_start|>在单位圆上的任意一点与原点的连线,该线在 xy 轴上的投影分别为$sinx,cosx$,根据勾股定理可证$sin^2x+cos^2x=1$<|im_end|>


[format prompt input_id]:
 [[151644, 198, 2, 46487, 25, 105043, 30709, 99949, 100857, 100168, 31914, 11, 14880, 99464, 100700, 102104, 20002, 13872, 43589, 86119, 198, 2, 6448, 21701, 15940, 61, 17, 87, 10, 9407, 61, 17, 87, 28, 5267, 2, 4939, 3846, 2821, 25, 151648, 59151, 17714, 57960, 79075, 90, 16, 31716, 151645, 198, 2, 6448, 25, 100678, 5267, 2, 4939, 3846, 2821, 25, 151648, 18493, 75317, 100213, 101913, 108112, 100380, 57218, 52129, 27442, 9370, 116539, 11, 75882, 43268, 18493, 30784, 8908, 121, 112, 101913, 111367, 105706, 3, 15940, 87, 11, 9407, 87, 54876, 100345, 105170, 99223, 22382, 21887, 30440, 33477, 3, 15940, 61, 17, 87, 10, 9407, 61, 17, 87, 28, 16, 3, 151645]]


[form

上述的 tokenizer 化出现了诡异的现象，`<|box_start|>` 应当被处理成 ID.151648, 但在 `[format prompt input_id]:` 序列里并无`151648` ID

结论是对文本格式化处理，再 tokenize, 我们所标记的 special token 可能会被编译成其他数据，结论是方式 1 不安全。

先对各 message 做 tokenize， 再格式化拼接。

## 方式2

对 token id 进行拼接


In [11]:
tokenizer(DEFINED_SOS_TOKEN).input_ids[0]

151644

In [12]:
def ChatTemplateToken(example, tokenizer):
    sos_token_id = tokenizer(DEFINED_SOS_TOKEN).input_ids[0] 
    eos_token_id = tokenizer(DEFINED_EOS_TOKEN).input_ids[0] 
    
    input_ids = [ sos_token_id ]
    is_labels = [ 0 ]
    
    for i, item in enumerate(example):
        if item['role'] == 'ASSISTANT':
            prompt = '\n#' + item['role'] + ':'
            content_prompt = item['content'] 
            prompt_token_ids = tokenizer(prompt).input_ids
            content_prompt = tokenizer(content_prompt).input_ids

            is_labels += [0]*len(prompt_token_ids) + [1]*len(content_prompt) + [1] # last [1] is eos 
            input_ids += prompt_token_ids + content_prompt + [eos_token_id]
            
        else:
            prompt = '\n#' + item['role'] + ':' + item['content'] 
            prompt_token_ids = tokenizer(prompt).input_ids
            input_ids += prompt_token_ids
            
            is_labels += [0]*len(prompt_token_ids)
            
    return input_ids, is_labels

input_ids, is_labels = ChatTemplateToken(messages_1, tokenizer)

print(input_ids, '\n\n')
print(is_labels, '\n\n')

format_prompt = tokenizer.decode(input_ids, skip_special_tokens=False)
print(format_prompt,)

[151644, 198, 2, 46487, 25, 105043, 30709, 99949, 100857, 100168, 31914, 11, 14880, 99464, 100700, 102104, 20002, 13872, 43589, 86119, 198, 2, 6448, 21701, 15940, 61, 17, 87, 10, 9407, 61, 17, 87, 19884, 198, 2, 4939, 3846, 2821, 25, 59151, 17714, 57960, 79075, 90, 16, 31716, 151645, 198, 2, 6448, 25, 100678, 30, 198, 2, 4939, 3846, 2821, 25, 18493, 75317, 100213, 101913, 108112, 100380, 57218, 52129, 27442, 9370, 116539, 11, 75882, 43268, 18493, 30784, 8908, 121, 112, 101913, 111367, 105706, 3, 15940, 87, 11, 9407, 87, 54876, 100345, 105170, 99223, 22382, 21887, 30440, 33477, 3, 15940, 61, 17, 87, 10, 9407, 61, 17, 87, 28, 16, 3, 151645] 


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 


<|im_start|>
#SY

## Dataset

Dataset 一般分 3 种

1. Raw-Data Dataset
2. Token-id Dataset
3. Batch token-id Dataset, 在 2 的基础上 batch 化处理，即要做 padding 等操作，这里的 padding 需要对 attention mask、label 同等处理

In [13]:
class RawSFTDataset:
    def __init__(self, messages):
        self.data = messages
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

dataset = RawSFTDataset(messages_list)
dataset[1]

[{'role': 'SYSTEM', 'content': '你是小冬瓜智能体,请安全详细回答用户 USER 的问题'},
 {'role': 'USER', 'content': '什么是人工智能?'},
 {'role': 'ASSISTANT', 'content': '人工智能是让机器模拟人类思维的技术。'}]

In [14]:
# 采用拼接 token_id 方法构造数据集

import torch
from torch.utils.data import Dataset, DataLoader

class TokenSFTDataset(Dataset):
    def __init__(self, messages_list, tokenizer):
        self.data = [ ChatTemplateToken(messages, tokenizer) for messages in messages_list ]
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return {
            'input_ids':self.data[idx][0],
            'is_label':self.data[idx][1]
        }

dataset = TokenSFTDataset(messages_list, tokenizer)
print(dataset[1]['input_ids'])
print(dataset[1]['is_label'])

print(tokenizer.decode(dataset[1]['input_ids']))

# 特别注意 <EOS> 一定是 label, 如果没有, 则会导致 SFT 模型生成时无法停止
print(tokenizer.decode(dataset[1]['input_ids'][-1]))

[151644, 198, 2, 46487, 25, 105043, 30709, 99949, 100857, 100168, 31914, 11, 14880, 99464, 100700, 102104, 20002, 13872, 43589, 86119, 198, 2, 6448, 25, 106582, 104455, 30, 198, 2, 4939, 3846, 2821, 25, 104455, 20412, 99258, 102182, 105717, 103971, 102141, 105535, 1773, 151645]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
<|im_start|>
#SYSTEM:你是小冬瓜智能体,请安全详细回答用户 USER 的问题
#USER:什么是人工智能?
#ASSISTANT:人工智能是让机器模拟人类思维的技术。<|im_end|>
<|im_end|>


至于第3种方法, batch化数据集不是一个好的策略，

1. 其所产生的 padding 数据增加存储占用
2. 好处在于在高速训练过程, 可以直接取batch数据传入到GPU去计算

更通用的方式是手写一个 collate function

## Data collate function

在深度学习训练过程，数据集会 shuffle, 多条随机数据成 batch

In [15]:
class TokenSFTDataset(Dataset):
    def __init__(self, messages_list, tokenizer):
        data_list = [ ChatTemplateToken(messages, tokenizer) for messages in messages_list ]
        self.data = []
        for data in data_list:
            self.data.append( 
                [torch.tensor(data[0], dtype=torch.long), 
                 torch.tensor(data[1], dtype=torch.long)]
            )
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        """
        token 返回数据一般是 tensor
        """
        return {
            'input_ids': self.data[idx][0],
            'is_label':self.data[idx][1]
        }

In [16]:
def paddding_collate_fn(batch_data, pad_token_id=None, ignore_index=-100):

    input_lens = []
    label_lens = []
    bs = len(batch_data)

    # padding longest maxlen
    for data in batch_data:
        input_lens.append(data['input_ids'].shape[0])
        max_input_len = torch.max(torch.tensor(input_lens, dtype=torch.long))
    
    # Right Padding
    input_ids = torch.ones(bs, max_input_len, dtype=torch.long) * pad_token_id
    attention_masks = torch.zeros(bs, max_input_len, dtype=torch.long)
    labels = torch.ones(bs, max_input_len, dtype=torch.long) * ignore_index

    for i in range(bs):
        input_ids[i, :input_lens[i]] = batch_data[i]['input_ids']
        attention_masks[i, :input_lens[i]] = 1
        
        idx = torch.where( batch_data[i]['is_label'] != 0)[0]
        labels[i, idx-1] = batch_data[i]['input_ids'][idx]

    return {
        'input_ids': input_ids,
        'attention_masks': attention_masks,
        'labels': labels,
    }

class PaddingCollateFunction:
    def __init__(self, pad_token_id: int, ignore_index):
        self.pad_token_id = pad_token_id
        self.ignore_index = ignore_index

    def __call__(self, batch) -> dict:
        batch = paddding_collate_fn(batch, self.pad_token_id, self.ignore_index )
        return batch

In [17]:
DEFINE_IGNORE_INDEX=-100

        
dataset = TokenSFTDataset(messages_list, tokenizer)
collate_fn = PaddingCollateFunction(pad_token_id=0, ignore_index=DEFINE_IGNORE_INDEX)
dataloader = DataLoader(dataset, 
                    batch_size=2, 
                    shuffle=False, # True
                    collate_fn = collate_fn) 

for i, batch in enumerate(dataloader):
    print(batch['input_ids'])
    print(batch['attention_masks'])
    print(batch['labels'])

    break  

tensor([[151644,    198,      2,  46487,     25, 105043,  30709,  99949, 100857,
         100168,  31914,     11,  14880,  99464, 100700, 102104,  20002,  13872,
          43589,  86119,    198,      2,   6448,  21701,  15940,     61,     17,
             87,     10,   9407,     61,     17,     87,  19884,    198,      2,
           4939,   3846,   2821,     25,  59151,  17714,  57960,  79075,     90,
             16,  31716, 151645,    198,      2,   6448,     25, 100678,     30,
            198,      2,   4939,   3846,   2821,     25,  18493,  75317, 100213,
         101913, 108112, 100380,  57218,  52129,  27442,   9370, 116539,     11,
          75882,  43268,  18493,  30784,   8908,    121,    112, 101913, 111367,
         105706,      3,  15940,     87,     11,   9407,     87,  54876, 100345,
         105170,  99223,  22382,  21887,  30440,  33477,      3,  15940,     61,
             17,     87,     10,   9407,     61,     17,     87,     28,     16,
              3, 151645],
  

# 细节：

Q1: 根据 prompt 写出 label？

| Prompt | 回   | 答   | :    | 瓜   | 哥   | 真   | 帅      | `<EOS>` |
| ------ | ---- | ---- | ---- | ---- | ---- | ---- | ------- | ------- |
| Label  | ? | ?| ?   | ?   | ?   |?   | ?| ?    |

A1: 

| Prompt | 回   | 答   | :    | 瓜   | 哥   | 真   | 帅      | `<EOS>` |
| ------ | ---- | ---- | ---- | ---- | ---- | ---- | ------- | ------- |
| Label  | -100 | -100 | 瓜   | 哥   | 真   | 帅   | `<EOS>` | -100    |

---

Q2:

对于token id 序列长度为 8 的输入, 其输出 logits 为 8x|V|, 那么有效的 logits 是哪些？

A2:

logits[2:-1], pos_id `2,3,4,5,6`

| Prompt | 回   | 答   | :    | 瓜   | 哥   | 真   | 帅      | `<EOS>` |
| ------ | ---- | ---- | ---- | ---- | ---- | ---- | ------- | ------- |
| Label  | -100 | -100 | 瓜   | 哥   | 真   | 帅   | `<EOS>` | -100    |
| pos_id  | 0 | 1 | 2   | 3   | 4   | 5   | 6 | 7    |


---

Q3:

在 pytorch 的 loss_fn 中, 如何忽略特定 label 的损失

A3:

`loss_fn = nn.CrossEntropyLoss(ignore_index = -100)`

# 总结

1. SFT 的数据要较多细节，建议阅读完本章节后用 pytorch 手写一遍 dataset、template、collate_fn、dataloader；
3. 在工程开发工程中，需要通过 pytorch 灵活处理数据集(如处理多轮对话)，如上述的 `labels` 实际上在模版函数上提前记录好 `is_label`
2. HuggingFace 的 Transformers 框架提供了一系列数据处理类，可以直接调用。由于 Transfomers 框架对 pytorch 的数据类再做了封装，定制 Transformers 的数据类实现较为繁琐。
3. Tokenizer 有较多坑，例如 special token 未编码成 1 个 token id；
4. 需要每一步 check 中间数据, 避免从后往前排查。